In [ ]:
import os, re, matplotlib, pandas, collections, importlib, sys, pickle, random, glob, dill, time
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from importlib import reload
from pathlib import Path


top = Path('/Users/dp/pma/dataAndScripts/clip/meta/')
top_dir = top

os.chdir(top)

sys.path.append('/Users/dp/pma/')
import sameRiver
import sameRiver.biotypeUtils
import sameRiver.stacked_bargraph
import sameRiver.readsPerGene
import sameRiver.exp
import sameRiver.rnaDataFileMaker

importlib.reload(sameRiver.readsPerGene)
importlib.reload(sameRiver.stacked_bargraph)

from sameRiver.readsPerGene import *
from sameRiver.stacked_bargraph import *

##################################################################
# 1. ann_counts is raw read counts. It also has biotypes added as the column 'Gene type'.



In [ ]:
import sameRiver.intronExonSplit
import sameRiver.statsForCountsNB
import sameRiver.biotypeDf
import sameRiver.stacked_bargraph
importlib.reload(sameRiver.biotypeUtils)
importlib.reload(sameRiver.countsO)
importlib.reload(sameRiver.stacked_bargraph)
importlib.reload(sameRiver.statsForCountsNB)
importlib.reload(sameRiver.readsPerGene)
from sameRiver.readsPerGene import *


import sameRiver.metadata.negative_metadata as negative_metadata
import sameRiver.metadata.positive_metadata_all
import sameRiver.negativeCounts
import sameRiver.positiveCounts
import sameRiver.scheme
import sameRiver.statsForCountsNB
importlib.reload(sameRiver.metadata.positive_metadata_all)
importlib.reload(sameRiver.positiveCounts)
importlib.reload(sameRiver.negativeCounts)
importlib.reload(sameRiver.statsForCountsNB)
importlib.reload(sameRiver.scheme)


In [ ]:
def numeric_columns(_df):
    return [x for x in _df.columns if (_df[x].dtype.kind in 'bifc')]


figname = lambda _str: '/Users/dp/pma/dataAndScripts/clip/figs/{}.pdf'.format(_str)


In [ ]:
import sameRiver.biotypeArtist
from pathlib import PurePath, Path

importlib.reload(sameRiver.biotypeUtils)
importlib.reload(sameRiver.biotypeArtist)
importlib.reload(sameRiver.stacked_bargraph)
importlib.reload(sameRiver.biotypeDf)

# Load data and globals.
#top = Path('/Users/dp/pma/dataAndScripts/clip/old_mapping_meta/')
#os.chdir(top)
print(f"Working in {top}...")

scheme = sameRiver.scheme.scheme(Path(top, 'scheme.xlsx'))

randos_rbps = [
    'FBL', 'Rbfox1', 'Rbfox2', 'hnRNPC', 
    'hnRNPD', 'CELF1', 'SF3B1', 'PCBP1',
    'UBA2', 'ETS2', 'CAPNS6', 'TPGS2', 
    'EPB41L5', 'CHMP3', 'CDK4', 'IDE', 
    'DCTN6', 'CCIN', 'ITPA', 
    'PCBP1 100P', 'PCBP1 100Q', 'PCBP1 dKH']
randos = [
    'UBA2', 'ETS2', 'CAPNS6', 'TPGS2', 
    'EPB41L5', 'CHMP3', 'CDK4', 'IDE', 
    'DCTN6', 'CCIN', 'ITPA', 
]
rbps = [
    'FBL', 'hnRNPC', 'SF3B1', 'PCBP1', 
    #'PCBP1:100P', 'PCBP1:100Q','PCBP1:dKH',
    'CELF1', 'Rbfox1', 'Rbfox2', 'hnRNPD',
    'KHDRBS2', 'A1CF', 'FUBP1', 'RPL5',#'CRNKL1',
    ]
# Select PCBP1s
pcbp1_names = ['PCBP1', 'PCBP1:100P', 'PCBP1:100Q', 'PCBP1:dKH', 'Gene type']

cfMuts = [
    'KHDRBS2', 'KHDRBS2:R168C', 'A1CF', 'A1CF:E34K', 'FUBP1', 'FUBP1:R429C', 'RPL5', 'RPL5:E82K',#'PCBP1', 'PCBP1:100Q'
]

#with open(Path(top, 'data/stats.dill'), 'rb') as f:
#    nb = dill.load(f)
# Write ./data/positive_counts_per_read.txt:
#nb.positives.save(write_txt=True, write_object=False)

# Load and filter positive counts_per_read
#df = pandas.read_csv('./data/positive_counts_per_read.txt', sep='\t', index_col=0)   
#df = nb.positives.counts_per_million_df.copy()
#df = sameRiver.biotypeUtils.biotype_filter(df)

# Load the read counts.
rpg = readsPerGene(f'{top}/counts.txt')
rpg.add_biotypes_column(
    reload=True,
    gtf_filename='/opt/genomes/repeats/repeats_and_genome.gtf',  # The newer, repeats/genome separate mapping. 
    #gtf_filename='/opt/genomes/repeats_and_ensembl_release94_GRCh38/combined_tsl1andNA.gtf' # The old mapping gtf. 
)
df = rpg.df

# Create a palette to be used for all figures.
type_list = [#'rRNA', #'ribozyme', #'7SK', 
             #'processed_transcript', 
             'snoRNA', #'nonsense_mediated_decay', 
             'tRNA', 'scaRNA', 
             'lincRNA',
             'snRNA', #'LTR',
             'protein_coding', 
             'repeat', #'intergenic'
            ]
type_colors = sns.cubehelix_palette(n_colors=len(type_list), start=.5, rot=-2)

# Lookup table of number (position in type list) -> color.
group_to_color = {_t:type_colors[m] for m, _t in enumerate(type_list)}


############################################################
# Fraction of all reads by biotype bargraph.
############################################################

def prepare_to_plot(df):
    
    df = df.copy()

    df = sameRiver.biotypeUtils.biotype_filter(df)
    print(f"From {set(df['Gene type'].tolist())},")
    
    bio = sameRiver.biotypeDf.biotypeDf(df=df)
    
    ok_biotypes = bio.only_biotypes_with_a_large_enough_fraction_of_some_dataset(
        minimum_percentage=3)
    
    print(f"biotypes with a large enough fraction: {ok_biotypes}")

    fraction_all_reads = bio.fraction_of_value_by_biotype_flat()
    fraction_all_reads.loc[:,'% reads (all RNAs)'] = fraction_all_reads['%']
    fraction_all_reads.loc[:,'Protein'] = [scheme.gene_from_fname(x) for x in fraction_all_reads['Protein']]
    fraction_all_reads = fraction_all_reads.groupby(['Protein', 'Gene type'])['% reads (all RNAs)'].mean().reset_index()

    sample_order = fraction_all_reads[fraction_all_reads['Gene type'] == 'protein_coding']
    sample_order.sort_values(by='% reads (all RNAs)', inplace=True)
    return fraction_all_reads, sample_order['Protein'].tolist()

def bargraphs(_df, proteins=[], name='', type_list=type_list):

    # Select proteins.
    df = _df.loc[:,[x for x in _df.columns if (scheme.gene_from_fname(x) in proteins) or x=='Gene type']]
    
    # Get rid of biotypes not in the type list.
    df = df.loc[[x in type_list for x in df['Gene type']], :]

    # Log what datasets were used.
    os.makedirs(f'{top}/logs/', exist_ok=True)    
    with open(f'{top}/logs/datasets_used_for_biotypes.py', 'a') as f:
        f.write(f'datasets_for_{name} = ' + str(list(_df.columns)) + '\n')

    # Define color from the type list.
    type_colors = sns.cubehelix_palette(n_colors=len(type_list), start=.5, rot=-2)

    # Lookup table of number (position in type list) -> color.
    group_to_color = {_t:type_colors[m] for m, _t in enumerate(type_list)}

    # Get rid of biotypes not in the type list.
    df_given_types = df.loc[[x in type_list for x in df['Gene type']], :]

    print(f"Before calling prepare to plot... {set(df['Gene type'].tolist())}")
    (fraction_all_reads, sample_order) = prepare_to_plot(df_given_types)

    sameRiver.stacked_bargraph.stacked_bargraph(
        df=fraction_all_reads, value_label='% reads (all RNAs)', sample_label='Protein',
        hue_label='Gene type', title='% reads (all RNAs)',
        sample_order=sample_order,#counts.proteins(),
        group_to_color=group_to_color,
        fname=figname(f'biotypes_{name}_all_rnas'))

    # rRNA vs non-rRNA, keeping the order the same.
    rRNA = _df.loc[:,[x for x in _df.columns if (scheme.gene_from_fname(x) in proteins) or x=='Gene type']]
    rRNA['Gene type'] = [
        {'rRNA': 'rRNA', 'intergenic': 'intergenic'}.get(x, 'non-rRNA') for x in rRNA['Gene type']]

    type_list = ['rRNA', 'non-rRNA']#, 'intergenic']
    type_colors = sns.cubehelix_palette(n_colors=len(type_list), start=.8, rot=-1.5)
    group_to_color = {_t:type_colors[m] for m, _t in enumerate(type_list)}
    
    rRNA = rRNA.loc[[x in type_list for x in rRNA['Gene type']], :]
    (fraction_all_reads, _) = prepare_to_plot(rRNA)
    
    sameRiver.stacked_bargraph.stacked_bargraph(
        df=fraction_all_reads, value_label='% reads (all RNAs)', sample_label='Protein',
        hue_label='Gene type', title='% reads (all RNAs)',
        sample_order=sample_order,#counts.proteins(),
        group_to_color=group_to_color,
        fname=figname(f'biotypes_{name}_rRNA'), fig_height=2)

    # Exon vs intron, keeping the order the same.
    ex = _df.loc[:,[x for x in _df.columns if (scheme.gene_from_fname(x) in proteins) or x=='Gene type']]
    # Only mRNA.
    ex = ex.loc[[x=='protein_coding' for x in ex['Gene type']], :]
    # Make the gene type column just the designation of intron or exon.
    ex['Gene type'] = [x.split('::')[-1] for x in ex.index]
    type_list = ['exon', 'intron']

    type_colors = sns.cubehelix_palette(n_colors=len(type_list), start=7, rot=-1.4)
    group_to_color = {_t:type_colors[m] for m, _t in enumerate(type_list)}
    
    ex = ex.loc[[x in type_list for x in ex['Gene type']], :]
    (fraction_all_reads, _) = prepare_to_plot(ex)
    
    print(fraction_all_reads)
    
    sameRiver.stacked_bargraph.stacked_bargraph(
        df=fraction_all_reads, value_label='% reads (all RNAs)', sample_label='Protein',
        hue_label='Gene type', title='% reads (all RNAs)',
        sample_order=sample_order,#counts.proteins(),
        group_to_color=group_to_color,
        fname=figname(f'biotypes_{name}_perc_intronic'), fig_height=2)
    
with open('./logs/datasets_used_for_biotypes.py', 'w') as f:
    f.write('# Written by biotypes.ipynb.')  # Empty the file so it can be appended to.

# Plot RBPs.
bargraphs(df, proteins=cfMuts, name='Mutants')
bargraphs(df, proteins=rbps, name='RBPs')
bargraphs(df, proteins=randos, name='non-RBPs')
bargraphs(df, proteins=pcbp1_names, name='PCBP1')


## Create a counts file of counts collapsed by biotype and perform statistics.

In [ ]:
import sameRiver.readsPerGene
importlib.reload(sameRiver.readsPerGene)
from sameRiver.readsPerGene import *
# Class for creating a new metadata namespace, if wanted. (Not used currently, I think.)
class Namespace:
    def __init__(self, **kwargs): self.__dict__.update(kwargs)

# Import object with file paths and protein names.
import sameRiver.metadata.negative_metadata as negative_metadata
import sameRiver.metadata.positive_metadata_all as positive_metadata

# Change the counts filename to be one collapsed by biotype.
positive_metadata.ann_counts_file = f"{top}/data/biotype_counts.txt"
negative_metadata.ann_counts_file = f"{top}/data/biotype_counts.txt"

# Load counts.
rpg = rawReadsPerGene(f'{top_dir}/counts.txt', scheme_filename=f'{top_dir}/scheme.xlsx')
rpg.add_biotypes_column(
    gtf_filename='/Users/dp/pma/repeats_and_genome.gtf', reload=True)

# Write a version of the counts file collapsed by biotype.
a = rpg.df.groupby('Gene type').apply(np.sum) # Series of Gene type -> {Col: samples, Rows: sums}
a = pandas.DataFrame(a)
a = a.loc[:, numeric_columns(a)]
a.to_csv(f'{top}/data/biotype_ann_counts.txt', sep='\t')

# Convert to per read.
rpm = readsPerMillion(rpg)

# Per million reads: collapse by gene type.
rpm.df = rpm.df.groupby('Gene type').sum()#
#print('rpm.groupby sum->', rpm.df.sum(axis=0))
rpm.df = rpm.df[numeric_columns(rpm.df)]
rpm.reads_per_million_df = rpm.df  # Refresh the synonym.
rpm.df.to_csv(f'{top_dir}/data/biotype_per_read.txt', sep='\t')

# FYI: to get a Series of Gene type -> array of {Columns = protein datasets. Index = RNAs}:
# a = rpg.df.groupby('Gene type').apply(np.array)

#xpp = xlsPerProtein(rpm, xl_rate_fname='/Users/dp/pma/percentCrosslinked.xlsx')

# Redefine the metadata objects to load the raw counts.
positive_metadata = sameRiver.metadata.positive_metadata_all
positive_metadata.ann_counts_file = f'{top_dir}/data/biotype_ann_counts.txt'
negative_metadata.ann_counts_file = f'{top_dir}/data/biotype_ann_counts.txt'

# Create counts objects of data collapsed by biotype and save.
negatives = sameRiver.negativeCounts.negativeCounts(negative_metadata, xl_rate_fname='/Users/dp/pma/percentCrosslinked.xlsx')
positives = sameRiver.positiveCounts.positiveCounts(positive_metadata, xl_rate_fname='/Users/dp/pma/percentCrosslinked.xlsx')
positives.save(write_object=True, write_txt=True)  # Note this over-writes the regular data files.

# Do stats and write to tables/
nb = sameRiver.statsForCountsNB.statsForCountsNB(
    negatives=negatives, positives=positives, data_dir=f'{positive_metadata.top_dir}/data/')
nb.calculate_pvalues(which='per_read', test_mode=False)
nb.write_pvals_single_file(which='per_read', outfname=f'{top}/tables/pvalues_per_read_biotypes.xlsx')
nb.calculate_pvalues(which='per_protein')
nb.write_pvals_single_file(which='per_protein', outfname=f'{top}/tables/pvalues_per_protein_biotypes.xlsx')

In [ ]:
rpg = rawReadsPerGene(f'{top_dir}/data/biotype_ann_counts.txt', scheme_filename=f'{top_dir}/scheme.xlsx')
rpm = readsPerMillion(rpg)

xpp = xlsPerProtein(rpm, xl_rate_fname='/Users/dp/pma/percentCrosslinked.xlsx')

aves = rpm.average_by_protein()
aves['s'] = aves.sum(axis=1)
aves = aves.sort_values(by='s').tail(8)
del aves['s']
df = aves

#df = pandas.read_csv(f'{top_dir}/data/biotype_per_read.txt', sep='\t', index_col=0)
df = df[numeric_columns(df)]
df.clip(lower=0.1, inplace=True)
df = np.log10(df)
#print(df)

fig = plt.figure()
g = sns.clustermap(df.T, yticklabels=True)#, vmax=1, row_cluster=False)
g.ax_row_dendrogram.set_visible(False)
g.ax_col_dendrogram.set_visible(False)
#fig.set_figheight(20)
plt.savefig(figname('reads_per_million_biotype_clustermap'))
#plt.show()
plt.clf(); plt.close()

In [ ]:
xpp = xlsPerProtein(rpm, xl_rate_fname='/Users/dp/pma/percentCrosslinked.xlsx')

aves = rpm.average_by_protein()
pos = ['hnRNPC', 'hnRNPD', 'Rbfox1', 'Rbfox2', 'FUBP1', 'KHDRBS2',
      'A1CF', 'CELF1', 'SF3B1', 'PCBP1', 'RPL5', 'FBL']
negative_aves = aves.loc[:, [x for x in negative_metadata.random_proteins if x in aves]]
positive_aves = aves.loc[:, [x for x in pos if x in aves]]
mean_negative_aves = aves.mean(axis=1)
aves['Negatives'] = mean_negative_aves.to_list()
a = aves.sort_values(by='Negatives').tail(4)['Negatives']

top_types = list(a.index)

aves = aves.loc[top_types, :]

fig = plt.figure()
for _type in top_types:
    neg_vals = negative_aves.loc[_type, :]/1E6
    pos_vals = positive_aves.loc[_type, :]/1E6
    print(neg_vals)
    bins = np.linspace(0, max(pos_vals)*1.2, 20)
    #plt.hist([neg_vals, pos_vals], label=['neg', 'pos'])
    plt.hist(neg_vals, bins, alpha=0.5, label='Negatives')
    plt.hist(pos_vals, bins, alpha=0.5, label='RBPs')# color='r')
    plt.title(_type)
    plt.legend()
    plt.show()
    plt.clf(); plt.close()

In [ ]:
??filter


In [ ]:
from scipy import stats

def make_stat_df(rpm: readsPerGene) -> pandas.DataFrame:
    
    aves = rpm.average_by_protein()
    negative_aves = aves.loc[:,[x for x in negative_metadata.random_proteins if x in aves]]

    mean_negative_aves = aves.mean(axis=1)
    aves['Negatives'] = mean_negative_aves.to_list()

    s = {}
    for prot in rpm.proteins(): 
        cols = rpm.columns_for_a_protein(prot)
        neg_vals = negative_aves
        for RNA in rpm.df.index:
            s.setdefault(RNA, {})
            pos = rpm.df.loc[RNA, cols]
            neg = negative_aves.loc[RNA, :]

            std_neg = np.std(neg)
            z = (pos.mean() - neg.mean())/np.std(neg)
            s[RNA][f'{prot} z'] = z

            t = stats.ttest_ind(a=pos, b=neg)
            #s[prot][RNA][f'{prot} pvalue'] = t.pvalue
            s[RNA][f'{prot} -log(pvalue)'] = -np.log10(t.pvalue)
            #print(prot, RNA, z, t)
    zt = pandas.DataFrame.from_dict(s, orient='index')
    return zt

#aves = rpm.average_by_protein()

xpp = xlsPerProtein(rpm, xl_rate_fname='/Users/dp/pma/percentCrosslinked.xlsx')

zt_pm = make_stat_df(rpm)
zt_pm.to_excel('temp_pm.xlsx')

zt_pp = make_stat_df(xpp)
zt_pp.to_excel('temp_pp.xlsx')

top_types = list(a.index)

fc = {}
for prot in set(['Negatives']) | rpm.proteins():
    fc[prot] = {x: aves[prot][x]/mean_negative_aves[x] for x in aves.index}
fc = pandas.DataFrame.from_dict(fc)

print(zt.head(2))

#fc.clip(lower=0.001, inplace=True)
#fc = np.log2(fc)
#print(fc.columns)


In [ ]:

# Initialize figure objects.

# Will only use these biotypes.
used_types = ['protein_coding', 'rRNA', 'snoRNA', 'repeat', 'snRNA']

# Define the list of proteins to be plotted.
protein_list = [x for x in positive_metadata.positive_proteins if x not in negative_metadata.random_proteins]
protein_list += negative_metadata.random_proteins
protein_list = [x for x in protein_list if x in rpm.proteins()]
protein_list = [x for x in protein_list if x not in (#'CCIN', 'EPB41L5', 
                                                     'DDX3X', 'RARS2',)]
protein_order = ['CDK4', 'CHMP3', 'DCTN6', 'ETS2', 'IDE', 'ITPA', 'TPGS2', 'UBA2', 'CCIN', 'EPB41L5',
                'SMAD3', 'SMAD4', 'BARD1', 'BRCA1', 'CRNKL1',
                'RPL5', 'KHDRBS2', 'FUBP1', 'A1CF', 'hnRNPD', 'Rbfox1', 'Rbfox2',
                'CELF1', 'PCBP1', 'SF3B1', 'hnRNPC', 'FBL']

protein_list = protein_order[::-1]
# For mapping biotype to color.
to_color = ['black', 'red', 'indigo', 'cyan', 'violet']

def plot_bubbles(rpm, zt, fig_fname, marker_params='per_read'):
    
    fig = plt.figure()
    ax = plt.subplot()
    ax.scatter([0], [0], c='w')

    for y_pos, protein in enumerate(protein_list):

        ax.axhline(y=y_pos, color='k', alpha=0.5, linewidth=0.5)

        for RNA in filter(lambda r: r in used_types, rpm.df.index):
            pval = zt.loc[RNA, f'{protein} -log(pvalue)']
            zscore = zt.loc[RNA, f'{protein} z']
            marker = 'o' if zscore > 0 else 'D'

            # Plot a color label only once.
            label = RNA if protein=='FBL' else None

            markersize = 15*np.log10(abs(zscore)) if marker_params == 'per_read' else 15*np.log10(abs(zscore))
            
            ax.plot(
                pval, [y_pos], marker, color=to_color[used_types.index(RNA)], markeredgewidth=0,
                alpha=0.5, markersize=markersize,
                label=label)

        ax.axvline(x=2, linestyle=':', color='k', alpha=0.1, linewidth=1)
        ax.set_yticks(range(len(protein_list)))
        ax.set_yticklabels(protein_list)

    plt.legend()
    fig.set_figheight(10)
    fig.savefig(fig_fname)
    plt.show(); plt.clf(); plt.close()

    ##################
    # Plot legend for bubble size interpretation.
    fig = plt.figure()
    ax = plt.subplot()
    ax.scatter([0],[0], c='w')
    
    zscores = [2, 5, 10, 100]
    pvals = [1, 3, 5, 7]
    
    for y_pos, protein in enumerate(protein_list):
#        for RNA in filter(lambda r: r in used_types, rpm.df.index):
        RNA = 'protein_coding'
        marker = random.choice(['o', 'D'])
        for pval, zscore in zip(pvals, zscores):
            markersize = 15*np.log10(abs(zscore)) if marker_params == 'per_read' else 15*np.log10(abs(zscore))
            ax.plot(
                pval, [y_pos], marker, color=to_color[used_types.index(RNA)], markeredgewidth=0,
                alpha=0.5, markersize=markersize,
                label=label)
                
    ax.axvline(x=2, linestyle=':', color='k', alpha=0.1, linewidth=1)
    ax.set_yticks(range(len(protein_list)))
    ax.set_yticklabels(protein_list)
        
    fig.set_figheight(10)
    fig.savefig(figname('legend_for_biotype_stripplot_significance_and_z_score'))
    plt.show(); plt.clf(); plt.close()    
    # End plot legend.
    ##################
    
plot_bubbles(rpm, zt_pm, figname('biotype_stripplot_significance_and_z_score_per_read'), marker_params='per_read')
plot_bubbles(xpp, zt_pp, figname('biotype_stripplot_significance_and_z_score_per_protein'), marker_params='per_protein')

protein_order = ['CDK4', 'CHMP3', 'DCTN6', 'ETS2', 'IDE', 'ITPA', 'TPGS2', 'UBA2', 'CCIN', 'EPB41L5',
                'SMAD3', 'SMAD4', 'BARD1', 'BRCA1', 'CRNKL1',
                'RPL5', 'KHDRBS2', 'FUBP1', 'A1CF', 'hnRNPD', 'Rbfox1', 'Rbfox2',
                'CELF1', 'PCBP1', 'SF3B1', 'hnRNPC', 'FBL']

In [ ]:
from pprint import pprint

scheme = sameRiver.scheme.scheme(Path(top, 'scheme.xlsx'))
df = rpm.df
print(len(df.index))

#pprint(df.sum())
cf_lists = [
    ('KHDRBS2', 'KHDRBS2-R168C'),
    ('A1CF', 'A1CF-E34K'),
    #('RPL5', 'RPL5-E82K'),
    #('FUBP1', 'FUBP1-R429C'),
    #('PCBP1', 'PCBP1-L100Q')
    #('CRNKL1', 'CRNKL1-S128F'),
    #('RARS2', 'RARS2-R6C'),
]

import scipy.stats as stats

changes = {}

for (wt, mut) in cf_lists:
    print(wt, mut)
    
    changes[wt] = {}
    wt_cols = [x for x in df.columns if wt in x and mut not in x]
    mut_cols = [x for x in df.columns if mut in x]

    wt_batches = [x.split('_')[0] for x in wt_cols]
    mut_batches = [x.split('_')[0] for x in mut_cols]
    batch_lookup = {batch_n: batch_str for batch_n, batch_str in enumerate(wt_batches)}
    
    def batch_cols(batch_n):
        wts = [x for x in wt_cols if x.split('_')[0]==batch_lookup[batch_n]]
        muts = [x for x in mut_cols if x.split('_')[0]==batch_lookup[batch_n]]
        return (wts, muts)
    
    for biotype in df.index:
        cols_paired = [batch_cols(n) for n in range(len(wt_batches))]
        vals = [(float(df.loc[biotype, wtcol]), float(df.loc[biotype, mutcol])) for (wtcol, mutcol) in cols_paired]
        
        if np.mean(vals) < 100:
            continue  # Skip categories with less than 0.01% of all signal.
            
        ratios = [_mut/_wt for _wt,_mut in vals if _wt>0]
        ttest = stats.ttest_1samp(ratios, popmean=1)

        if len(ratios):
            changes[wt][biotype] = {
                'log2vsWT': np.log2(np.mean(ratios)),
                '-log10(P value)': -np.log10(ttest.pvalue),
                'WT mean RPM': np.mean(vals),
                'Gene type': biotype}
            
    changes[wt] = pandas.DataFrame(changes[wt]).T

# Select PCBP1s
pcbp1_names = ['PCBP1', 'PCBP1:100P', 'PCBP1:100Q', 'PCBP1:dKH', 'Gene type']

# Create a palette to be used for all figures.
type_list = ['rRNA', #'ribozyme', #'7SK', 
             'processed_transcript', 'snoRNA', #'nonsense_mediated_decay', 
             'tRNA', 'scaRNA', 
             'lincRNA',
             'snRNA', #'LTR',
             'protein_coding', 'repeat', #'intergenic'
            ]


type_colors = sns.cubehelix_palette(n_colors=len(type_list), start=.5, rot=-2)
protein_colors = sns.cubehelix_palette(n_colors=len(cf_lists), start=.5, rot=-2)

def ploty(log2vsWT, pair, axes):
    p1 = plt.scatter(
        log2vsWT['log2vsWT'], log2vsWT['-log10(P value)'], #'.',
        color=protein_colors[cf_lists.index(pair)], alpha=0.5, s=100,
        label=pair[1], edgecolors=None, linewidths=0)
    axes.append(p1)
    #axes[-1].set_label(pair[1])

    # Add text labels to outliers on the plot.
    for line in range(0,log2vsWT.shape[0]):
        if log2vsWT['-log10(P value)'][line] < 2:
            continue  # Don't label P>0.01 dots.
        if abs(log2vsWT['log2vsWT'][line]) < np.log2(1.1):
            continue  # Don't label FC < 1.5

        plt.text(log2vsWT['log2vsWT'][line]+0.01, log2vsWT['-log10(P value)'][line], 
             log2vsWT['Gene type'][line], horizontalalignment='left', 
             size='small', color='black')#, weight='semibold')

    return p1

axes = []
for (wt, mut) in cf_lists:
    print(wt, '---->')
    p1 = ploty(changes[wt], (wt, mut), axes)

plt.legend(axes, [pair[1] for pair in cf_lists])
plt.show()
plt.clf(); plt.close()

In [ ]:
import pandas
import sameRiver
import numpy as np
import sameRiver.biotypeDf
import sameRiver.biotypeUtils
import sameRiver.intronExonSplit
import sameRiver.biotypeArtist
import sameRiver.biotypeAdder
importlib.reload(sameRiver.biotypeDf)
importlib.reload(sameRiver.biotypeArtist)
importlib.reload(sameRiver.biotypeUtils)
#importlib.reload(sameRiver.countsO)
importlib.reload(sameRiver.stacked_bargraph)

# Create a palette to be used for all figures.
type_list = [#'rRNA', #'ribozyme', #'7SK', 
             #'processed_transcript', 
             'snoRNA', #'nonsense_mediated_decay', 
             'tRNA', 'scaRNA', 
             'lincRNA',
             'snRNA', #'LTR',
             'protein_coding', 
    #'SINE/Alu', 'LINE/L1',
    'Repetitive element',
            #'repeat', #'intergenic'
            ]
type_colors = sns.cubehelix_palette(n_colors=len(type_list), start=.5, rot=-2)

# Lookup table of number (position in type list) -> color.
group_to_color = {_t:type_colors[m] for m, _t in enumerate(type_list)}

##################################################################
# Bargraphs of target RNA numbers.
##################################################################

# Load tools and data.
top = '/Users/dp/pma/dataAndScripts/clip/old_mapping_meta/'
#with open(f'{top}/data/stats.dill', 'rb') as f:
#    nb = dill.load(f)

rpg = readsPerGene(filename=f'{top}/ann_counts.txt', scheme_filename=f"{top}/scheme.xlsx")

raw_read_df = sameRiver.combineReplicates.combine_protein_replicates(rpg.df, nb, scheme)
#print(df.loc['>0_11_Crp#LINE/CR1::exon'])

    
def raw_cutoff(x):
    if x < 5:
        return 0
    return x

for col in rpg.numeric_columns():
    rpg.df[col] = [raw_cutoff(v) for v in rpg.df[col]]

def rep(_str):
    if '/' in _str:
        return 'Repetitive element'
    return _str

rpg.df['Gene type'] = [rep(_) for _ in rpg.df['Gene type']]
rpm = readsPerMillion(rpg)
xpp = xlsPerProtein(rpm, xl_rate_fname='/Users/dp/pma/percentCrosslinked.xlsx')

scheme = sameRiver.scheme.scheme('./scheme.xlsx')

#ba = sameRiver.biotypeAdder.biotypeAdder.load()

# Per read or per protein
which = 'per_protein'

# Get the read counts table we'll be using.
if which == 'per_read':
    df = rpm.df#nb.positives.counts_per_million_df.copy()
    pvals = pandas.read_excel(f"{top}/tables/pvals_per_read.xlsx", index_col=0)
    #pandas.read_csv(f"{top}/tables/pvalues_per_read.csv", sep='\t', index_col=0)
elif which == 'per_protein':
    df = xpp.df#nb.positives.counts_per_protein_df.copy()
    pvals = pandas.read_excel(f"{top}/tables/pvals_per_protein.xlsx", index_col=0)#pandas.read_csv(f"{top}/tables/pvalues_per_protein.csv", sep='\t', index_col=0)

print(f"Applying a raw read cutoff by setting pvalues to 1.0 if not enough raw reads.")
for col in pvals:
    pvals[col] = [(p if raw_read_df.loc[gene, col]>9 else 1.) for gene, p in zip(pvals.index, pvals[col])]

# Only biotypes in type list.
df = df.loc[[(x in type_list) for x in df['Gene type']], :]

# Combine protein replicates:
import sameRiver.combineReplicates
df = sameRiver.combineReplicates.combine_protein_replicates(df, nb, scheme)

# Convert the read table of reads by RNA into three dataframes, all by biotype.
[reads_in_targs_df, n_targets_df, all_reads_df] = sameRiver.biotypeArtist.dataframe_of_target_biotypes(
    df, pvals, scheme, which=which, p_val_to_define_targets=1E-4)

print('----=----'* 7)
#print(f"all_reads_df={all_reads_df.head()}")
all_reads_df = sameRiver.biotypeUtils.biotype_filter(all_reads_df)
mrna = all_reads_df[all_reads_df['Gene type'] == 'protein_coding']
to_prot = dict(zip(mrna['Protein'], mrna['% reads (all RNAs)']))
by_mrna = sorted(to_prot.keys(), key=lambda x: to_prot[x])
#by_mrna = filter_proteins(by_mrna)
#print(f'pizza: after returning: n_targets_df={n_targets_df}')

x = n_targets_df.groupby(by=['Protein']).sum()
#print(f'x={x.head(5)}')
by_target_number = x.sort_values(by=['# RNAs'], ascending=False, inplace=False).index

to_remove = [ 'PCBP1 100P', 'PCBP1 100Q', 'PCBP1 dKH', 'HCT116']
pcbp1s = ['PCBP1', 'PCBP1:100P', 'PCBP1:100Q', 'PCBP1:dKH']
#by_target_number = pcbp1s
#by_target_number = [x for x in by_target_number if (re.sub(':', ' ', x) not in to_remove)]
#by_target_number = sorted(to_num.keys(), key=lambda x: to_num[x])
randos_rbps = [
    'FBL', 'Rbfox1', 'Rbfox2', 'hnRNPC', 
    'hnRNPD', 'CELF1', 'SF3B1', 'PCBP1',
    'UBA2', 'ETS2', 'CAPNS6', 'TPGS2', 
    'EPB41L5', 'CHMP3', 'CDK4', 'IDE', 
    'DCTN6', 'CCIN', 'ITPA', 
    'PCBP1 100P', 'PCBP1 100Q', 'PCBP1 dKH']
randos = [
    'UBA2', 'ETS2', 'CAPNS6', 'TPGS2', 
    'EPB41L5', 'CHMP3', 'CDK4', 'IDE', 
    'DCTN6', #'CCIN', 
    'ITPA', 
]
rbps = [
    'FBL', 'hnRNPC', 'SF3B1', 'PCBP1', 
    #'PCBP1:100P', 'PCBP1:100Q','PCBP1:dKH',
    'CELF1', 'Rbfox1', 'Rbfox2', 'hnRNPD',
    'A1CF', 'KHDRBS2', 'FUBP1', 'RPL5',
    ]
chosen_columns = randos + rbps#rbps
print(f"pizza: by_target_number={by_target_number}")
print(f"pizza: chosen_columns = {chosen_columns}")
print(f'pizza: by_target_number={by_target_number}')
col_order =  [x for x in by_target_number if (x in chosen_columns)]
print(f"pizza: col_order={col_order}")
#randos_rbps_to_plot = [x for x in by_mrna if x in randos_rbps]
#print(randos_rbps_to_plot)
            
def ploty(protein_order, figprefix='biotypes_bargraph_'):
    sameRiver.stacked_bargraph.stacked_bargraph(
        df=all_reads_df, value_label='% reads (all RNAs)', sample_label='Protein', 
        hue_label='Gene type', title='% reads (all RNAs)',
        sample_order=protein_order,#counts.proteins(), 
        group_to_color=group_to_color,
        fname=figname('biotypes_' + figprefix + 'all_rnas'))

    print(f'pizza: right before plotting: n_targets_df={n_targets_df}')
    sameRiver.stacked_bargraph.stacked_bargraph(
        df=n_targets_df, value_label='# RNAs', sample_label='Protein', 
        hue_label='Gene type', title='# target RNAs',
        sample_order=protein_order,#counts.proteins(), 
        group_to_color=group_to_color,
        fname=figname('biotypes_' + figprefix + 'number_of_targets'))

    sameRiver.stacked_bargraph.stacked_bargraph(
        df=reads_in_targs_df, value_label='% reads (target RNAs)', sample_label='Protein', 
        hue_label='Gene type', title='% reads (in target RNAs)',
        sample_order=protein_order,#counts.proteins(), 
        group_to_color=group_to_color,
        fname=figname('biotypes_' + figprefix + 'target_rnas'))

#ploty(randos_rbps_to_plot, 'randos_')


#pcbp1s = [x for x in by_mrna if x in ['PCBP1', 'PCBP1 100P', 'PCBP1 100Q', 'PCBP1 dKH']]

#ploty(pcbp1s, 'pcbp1_')

ploty(col_order, figprefix=which)
#print(pvals['CDK4'])



In [ ]:
"""
hnRNP C bound mostly mRNA, while FBL was almost entirely crosslinked to snoRNA and rRNA (Figure 5E). 
"""


import sameRiver
import sameRiver.biotypeUtils
import sameRiver.intronExonSplit
importlib.reload(sameRiver.biotypeUtils)
#sameRiver.biotypeUtils.make_enst_gene_name_biotype_map_file(
#    gtf_filename='/opt/genome/Homo_sapiens.GRCh38.83/exon_CDS_ensembl_havana_only_Homo_sapiens.GRCh38.83_plus_snos_and_scar_from_ensembl_release91.gtf',
#                         )

#df = pandas.read_csv(
#    '/Users/dfporter/pma/dataAndScripts/clip/miseq/meta/ann_counts.txt', sep='\t',
#    index_col=0)
kk = '''
_scheme = sameRiver.scheme.scheme('/Users/dfporter/pma/dataAndScripts/clip/miseq/meta/scheme.xls')

counts = sameRiver.countsO.countsO(filename='/Users/dfporter/pma/dataAndScripts/clip/miseq/meta/ann_counts.txt',
                            scheme=_scheme,
                            index_col=0,
                            indexing_col_name='gene_name')
'''

#bio = sameRiver.biotypeUtils.biotyper(counts)
#bio.counts_df = counts.counts_df.copy()  # Just for safety

#df = df[[x for x in df.columns if not re.search(':', x)]].copy()
#print(df.columns)
_types_df = bio.fraction_by_biotype()
#print(_types_df.columns)
_types_df = _types_df[[x not in [
    #'PCBP1', 
    '100P', '100Q',
    #'SF3B1',
    'SF3B1:K700E',
    'Unknown1', 'Unknown2', 
    #'No vector',
#    'FBL', 'hnRNPC'
    'HA-only',
    'CSRP2',
    'CAPNS6',
    'hnRNPC:50k', 'hnRNPC:500cag', 
    'hnRNPC:4500agt', 'hnRNPC:500aac', 'hnRNPC:4500cag', 
    'hnRNPC:500agt', 'hnRNPC:0', 'hnRNPC:4500aac',
    'hnRNPC:500tca', 'hnRNPC:4500tca',
    ] for x in _types_df.Protein]]

protein_order = counts.proteins()
mrna = _types_df[_types_df['Gene type'] == 'protein_coding']
to_prot = dict(zip(mrna['Protein'], mrna['Reads (% total)']))
by_mrna = sorted(to_prot.keys(), key=lambda x: to_prot[x])
introns_perc = sameRiver.intronExonSplit.intronExonSplit().percent_introns(counts, by_mrna)
print(by_mrna)
print(introns_perc)

stacked_bargraph(df=_types_df, xlabel='Reads (% total)', ylabel='Protein', group_label='Gene type', 
                 protein_order=by_mrna)
fig = plt.figure()
sns.barplot(data=introns_perc, y='% Intronic', x='Protein', color='k')
plt.xticks(rotation=90)
fig.savefig('percent_intronic.pdf')
plt.show()

introns_perc['pre-mRNA score'] = [a*b for (a,b) in zip(
    introns_perc['% Intronic'], introns_perc['Total (log10)'])]
fig = plt.figure()
sns.barplot(data=introns_perc, y='Total (log10)', x='Protein', color='k')
plt.xticks(rotation=90)
plt.ylabel('Reads in mRNA (log10)')
fig.savefig('reads_in_mrna.pdf')

fig = plt.figure()
sns.barplot(data=introns_perc, y='pre-mRNA score', x='Protein', color='k')
plt.xticks(rotation=90)
plt.ylabel('pre-mRNA score')
fig.savefig('reads_in_mrna.pdf')

fig = plt.figure()
sns.barplot(data=introns_perc, y='% Intronic (- Back.)', x='Protein', color='k')
plt.xticks(rotation=90)
fig.savefig('percent_intronic.pdf')
plt.show()

introns_perc['pre-mRNA score (- Back.)'] = [a*b for (a,b) in zip(
    introns_perc['% Intronic (- Back.)'], introns_perc['Total (log10) (- Back.)'])]
fig = plt.figure()
sns.barplot(data=introns_perc, y='Total (log10) (- Back.)', x='Protein', color='k')
plt.xticks(rotation=90)
fig.savefig('reads_in_mrna.pdf')

fig = plt.figure()
sns.barplot(data=introns_perc, y='pre-mRNA score (- Back.)', x='Protein', color='k')
plt.xticks(rotation=90)
plt.ylabel('pre-mRNA score (- Back.)')
fig.savefig('reads_in_mrna.pdf')

In [ ]:

#make_gene2refseq2symbol_human()
#make_refseq_to_biotype()

def add_gene_name(df, refseq_name='/opt/genome/refseq/biotype_refseq_geneid_symbol.txt'):
    return add_gene_name_and_type(df, refseq_name=refseq_name, add_type=False)


#df = pandas.read_excel('/Users/dfporter/pma/miseq/Runs/170830/tables/counts.xls')
df = pandas.read_csv(
    '/Users/dfporter/pma/dataAndScripts/clip/miseq/meta/ann_counts.txt', sep='\t',
    index_col=0)

#make_biotype_map_file()
#add_biotypes_column(df)
#add_gene_name_and_type_from_refseq(df)
df.sort_index(by=df.columns[0], ascending=False, inplace=True)
#print(df['Gene type'].value_counts())
#print(df.columns)
#df.to_excel('/Users/dfporter/pma/miseq/Proteins/counts.xls')
print(df.head(1))

In [ ]:
def take_average_of_replicate_set(df, gene_name=None, replicate_columns=None):
    #print(df.columns)
    
    if type(gene_name)==type(''):
        replicate_columns = []
        for col in find_hits_columns(df):
            if re.search(gene_name, col):#.split('_')[0]):
                replicate_columns.append(col)
    
    #print(replicate_columns)
    if type(replicate_columns) == type([]):
        sub = df[replicate_columns].copy()
        _sum = sub.sum(axis=1)
        _sum = [x/max(len(replicate_columns), 1) for x in _sum]
        #print(sub.head(3))
        return _sum
    print("Some error in taking average of replicates, returning a column of zeros.")
    return [0] * len(df.index)

def take_average_of_replicates_for_multiple_genes(df, genes='All'):
    _results = {}
    rnas = df.index.tolist()
    
    cols = set([x.split('_')[2] for x in find_hits_columns(df) if len(x.split('_')) > 2])
    print(cols)
    
    for gene in cols:
        averages = take_average_of_replicate_set(df, gene_name=gene)
        _results[gene +'_average'] = dict(zip(rnas, averages))
        
    print(_results.keys())
    averaged_df = pandas.DataFrame.from_dict(_results)
    averaged_df['Gene'] = rnas

    if 'Gene type' in df.columns:
        averaged_df['Gene type'] = df['Gene type'].tolist()
    
    return averaged_df

ave_df = take_average_of_replicates_for_multiple_genes(df)
print(ave_df.head(2))


In [ ]:
import sameRiver
reload(sameRiver.stacked_bargraph)
#from sameRiver.stacked_bargraph import *
def purge_no_feature(df):
    def is_ok(x):
        if x == '_no_feature':
            return False
        if x == '_ambiguous':
            return False
        return True
    df['OK'] = [is_ok(n) for n in df['Gene'].tolist()]
    print("Purging rows that are not clearly assigned to a gene: {0}".format(df['OK'].value_counts()))
    df = df[df['OK']].copy()
    del df['OK']
    return df

def purge_columns_with_low_counts(df, min_count=100):
    hits_columns = find_hits_columns(df)
    to_del = set()
    for col in hits_columns:
        col_sum = df[col].sum()
        if col_sum <= min_count:
            to_del.add(col)
    print("Purging columns with less than {0} counts total:\n{1} out of {2} removed.".format(
        min_count, len(to_del), len(df.columns)))
    for col in to_del:
        del df[col]
    return df

def total_reads_per_protein(df):
    hits_columns = find_hits_columns(df)
    totals = collections.defaultdict(int)
    for col in hits_columns:
        gene = col.split('_')[0]
        totals[gene] += df[col].sum()
    return totals

def total_reads_in_dataframe(df):
    hits_columns = find_hits_columns(df, verbose=False)
    total = 0
    for col in hits_columns:
        total += df[col].sum()
    return total

def _genes(df):
    hits_columns = find_hits_columns(df)
    return set([x.split('_')[0] for x in hits_columns])

def _cols_of_gene(df, gene):
    cols_to_keep = set()
    hits_columns = find_hits_columns(df, verbose=False)
    for col in hits_columns:
        if col.split('_')[0] == gene:
            cols_to_keep.add(col)
    return list(cols_to_keep)

def biotypes_with_enough_reads(df, cutoff=1000):
    biotypes_to_keep = set()
    hits_columns = find_hits_columns(df)
    for biotype in set(df['Gene type'].tolist()):
        all_for_type = 0
        sub = df[df['Gene type']==biotype]
        for col in hits_columns:
            all_for_type += sub[col].sum()
        if all_for_type >= cutoff:
            biotypes_to_keep.add(biotype)
    print("Biotypes with at least {0} reads across all datasets (Ignoring 'Unknown'): {1}.".format(
        cutoff, len(biotypes_to_keep - set(['Unknown']))))
    return biotypes_to_keep - set(['Unknown'])

def fraction_by_biotype(df):
    for_df = []
    usable_types = biotypes_with_enough_reads(df)
    sub = df.copy()
    sub['keep'] = [(x in usable_types) for x in df['Gene type']]
    sub = sub[sub['keep']]
    del sub['keep']
    totals = total_reads_per_protein(sub)
    for biotype in biotypes_with_enough_reads(df):
        print(biotype)
        sub = df[df['Gene type']==biotype]
        for gene in _genes(df):
            this_gene = sub[_cols_of_gene(sub, gene)]
            reads_in_biotype = total_reads_in_dataframe(this_gene)
            for_df.append({'Gene type': biotype, 'Protein': gene,
                           'Reads': reads_in_biotype,
                        'Reads (% total)': 100 * reads_in_biotype/totals[gene]})
    return pandas.DataFrame(for_df)

def classify_a_gene_name(x, _type):
    if re.search('SNORD', x):
        return 'C/D box snoRNA'
    if re.search('SNORA', x):
        return 'H/ACA box snoRNA'
    if re.search('SCARNA', x):
        return 'scaRNA'
    if re.search('RNU', x):
        return 'snRNA'

    if _type == 'snRNA':
        return 'snRNA'
    if _type == 'protein_coding':
        return 'protein_coding'
    if _type == 'lincRNA':
        return 'lincRNA'
    return 'Unknown'

def classify_gene_names(df):
    #name, _type = zip(df['Gene name'].tolist(), df['Gene type'].tolist())
    df['Class'] = [classify_a_gene_name(gene_name, gene_type) for (gene_name, gene_type) in \
                  zip(df.index, df['Gene type'].tolist())]
    return df
#df = pandas.read_excel('/Users/dfporter/pma/miseq/Proteins/counts.xls')
#                    sep='\t')#, index_col=False)
#print(df)

df = ave_df.copy()

df = classify_gene_names(df)
#print(df)


for_pie = df[df['Class']!='Unknown']
for_pie = for_pie[for_pie['Class']!='protein_coding']
print
#sums = for_pie.hnRNPC_171104_CGATTA_AAC.groupby(for_pie.Class).sum()

#sums = for_pie.FBL.groupby(for_pie.Class).sum()
add_gene_name_and_type_from_refseq(df)
add_gene_name_and_type_from_refseq(for_pie)
#print(df.columns)
#print('====')
nc_df = [collections.defaultdict(dict)]
#hm = '''
for gene in ['hnRNPC', 'FBL', 'PCBP1', '100Q', '100P', 'UBA2', 'TPGS2', 'ITPA', 'CDK4', 'NoVector',
            'PCBP1:WT', 'PCBP1:100Q', 'PCBP1:100P']:
    gene += '_average'
    if gene not in for_pie:
        continue
    #print(gene, '->')
    sums = for_pie[gene].groupby(for_pie.Class).sum()
    sums = sums.to_dict()
#    sums['Protein'] = gene
#    sums['Gene'] = gene
    for _type, _reads in sums.items():
        row = {}
        row['Gene type'] = _type
        row['Reads'] = _reads
        row['Protein'] = gene
        row['Reads (% total)'] = _reads/max(1, for_pie[gene].sum())
        nc_df.append(row)
    #print(sums)
    #nc_df['']
    #plt.axis('equal')
    #plt.pie(sums, labels=sums.index)
    #plt.title(gene.split('_')[0])
    #plt.gcf().savefig('figs/ncRNA_type_pie_{0}.svg'.format(gene.split('_')[0]))
    #plt.show()
    #plt.clf()
#'''
nc_df = pandas.DataFrame(nc_df)
#find_hits_columns(df)
nc_df = purge_columns_with_low_counts(nc_df)
nc_df = nc_df.dropna(axis=0, how='any')
#df = purge_no_feature(nc_df)
print(nc_df)
#types_df = fraction_by_biotype(df)
protein_order = ['hnRNPC', 'FBL', 'SF3B1', 'SF3B1:K700E', 
                 'PCBP1', 'PCBP1:100Q','PCBP1:100P',
                'PCBP1:WT', '100Q', '100P',
                'UBA2', 'TPGS2', 'ITPA', 'CDK4', 'NoVector']
                
nc_df['Protein'] = [x.split('_')[0] for x in nc_df.Protein]
sameRiver.stacked_bargraph.stacked_bargraph(
    df=nc_df, xlabel='Reads (% total)', ylabel='Protein', group_label='Gene type', 
                 #protein_order=protein_order
                                           )
#print(types_df)


In [ ]:

#protein_order = ['SF3B1', 'SF3B1:K700E']
protein_order = ['hnRNPC', 'FBL', 'SF3B1', 'SF3B1:K700E', 'PCBP1:WT', 'PCBP1:100Q','PCBP1:100P',
                'PCBP1', '100P', '100Q', 'UBA2', 'TPGS2', 'ITPA', 'CDK4', 'NoVector']
#protein_order = ['NoVector', 'UBA2', 'TPGS2', 'ITPA', 'CDK4']
#protein_order = ['PCBP1', '100Q', '100P']
#protein_order = ['hnRNPC', 'FBL']
#df = pandas.read_excel('/Users/dfporter/pma/miseq/Proteins/counts.xls')

#df = purge_columns_with_low_counts(df)
#df = purge_no_feature(df)
print(df)
add_gene_name_and_type_from_refseq(df)
_types_df = fraction_by_biotype(df)

#types_df['keep'] = [x in ['FBL', 'hnRNPC'] for x in types_df.Protein]
#types_df = types_df.loc[types_df['keep']]
#del types_df['keep']
print(_types_df)
#protein_order = [#'hnRNPC', 'FBL', 'SF3B1', 'SF3B1:K700E', 
#                 'PCBP1', 'PCBP1:100Q','PCBP1:100P',
#                'PCBP1', '100Q', '100P'
#                ]

stacked_bargraph(df=_types_df, xlabel='Reads (% total)', ylabel='Protein', group_label='Gene type', 
                 protein_order=protein_order)

In [ ]:
#?sns.color_palette
sns.palplot(sns.light_palette('black'))
plt.show()
plt.clf()
#sns.choose_colorbrewer_palette(, as_cmap=True, data_type='s')


